In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from nltk.corpus import stopwords
import re
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

In [4]:
tweets = pd.read_csv("Tweets.csv")

In [6]:
tweets.airline_sentiment.value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [5]:
def convert_Sentiment(sentiment):
    if sentiment == "positive":
        return 2
    elif sentiment == "neutral":
        return 1
    elif sentiment == "negative":
        return 0


tweets.airline_sentiment = tweets.airline_sentiment.apply(
    lambda x: convert_Sentiment(x)
)
tweets

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,1,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,2,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,1,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,0,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,0,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,2,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,0,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,1,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,0,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)


In [6]:
tweets["text"] = tweets["negativereason"].fillna("") + " " + tweets["text"]
tweets

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,1,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,2,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials ...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,1,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I ...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,0,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,Bad Flight @VirginAmerica it's really aggressi...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,0,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,Can't Tell @VirginAmerica and it's a really bi...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,2,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different ...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,0,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,Customer Service Issue @AmericanAir leaving ov...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,1,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines t...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,0,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,Customer Service Issue @AmericanAir you have m...,NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)


In [7]:
def remove_twitter_username(text):
    return re.sub("@[^\s]+", "", text)


tweets["text"] = tweets["text"].apply(lambda x: remove_twitter_username(x))
tweets.text

0                                              What  said.
1          plus you've added commercials to the experie...
2          I didn't today... Must mean I need to take a...
3        Bad Flight  it's really aggressive to blast ob...
4        Can't Tell  and it's a really big bad thing ab...
                               ...                        
14635      thank you we got on a different flight to Ch...
14636    Customer Service Issue  leaving over 20 minute...
14637      Please bring American Airlines to #BlackBerry10
14638    Customer Service Issue  you have my money, you...
14639      we have 8 ppl so we need 2 know how many sea...
Name: text, Length: 14640, dtype: object

In [8]:
def remove_url(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)


tweets["text"] = tweets["text"].apply(lambda x: remove_url(x))
tweets.text

0                                              What  said.
1          plus you've added commercials to the experie...
2          I didn't today... Must mean I need to take a...
3        Bad Flight  it's really aggressive to blast ob...
4        Can't Tell  and it's a really big bad thing ab...
                               ...                        
14635      thank you we got on a different flight to Ch...
14636    Customer Service Issue  leaving over 20 minute...
14637      Please bring American Airlines to #BlackBerry10
14638    Customer Service Issue  you have my money, you...
14639      we have 8 ppl so we need 2 know how many sea...
Name: text, Length: 14640, dtype: object

In [9]:
def remove_emoji(text):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emojis with faces
        "\U0001F300-\U0001F5FF"  # emojis like 100
        "\U0001F680-\U0001F6FF"  # map emojis
        "\U0001F1E0-\U0001F1FF"  # flags emojis
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", text)


tweets["text"] = tweets["text"].apply(lambda x: remove_emoji(x))
tweets.text

0                                              What  said.
1          plus you've added commercials to the experie...
2          I didn't today... Must mean I need to take a...
3        Bad Flight  it's really aggressive to blast ob...
4        Can't Tell  and it's a really big bad thing ab...
                               ...                        
14635      thank you we got on a different flight to Ch...
14636    Customer Service Issue  leaving over 20 minute...
14637      Please bring American Airlines to #BlackBerry10
14638    Customer Service Issue  you have my money, you...
14639      we have 8 ppl so we need 2 know how many sea...
Name: text, Length: 14640, dtype: object

In [10]:
def abbreviate(text):
    text = re.sub(r"lol", " laughing out loud", text)
    text = re.sub(r"jk", " just kidding", text)
    text = re.sub(r"lmao", " laughing", text)
    text = re.sub(r"won\'t", " will not", text)
    text = re.sub(r"won\'t've", " will not have", text)
    text = re.sub(r"can\'t", " can not", text)
    text = re.sub(r"don\'t", " do not", text)
    text = re.sub(r"can\'t've", " can not have", text)
    text = re.sub(r"ma\'am", " madam", text)
    text = re.sub(r"let\'s", " let us", text)
    text = re.sub(r"ain\'t", " am not", text)
    text = re.sub(r"shan\'t", " shall not", text)
    text = re.sub(r"sha\n't", " shall not", text)
    text = re.sub(r"o\'clock", " of the clock", text)
    text = re.sub(r"y\'all", " you all", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"n\'t've", " not have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'d've", " would have", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ll've", " will have", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'re", " are", text)
    return text


tweets["text"] = tweets["text"].apply(lambda x: abbreviate(x))
tweets.text

0                                              What  said.
1          plus you have added commercials to the exper...
2          I did not today... Must mean I need to take ...
3        Bad Flight  it is really aggressive to blast o...
4        Ca not Tell  and it is a really big bad thing ...
                               ...                        
14635      thank you we got on a different flight to Ch...
14636    Customer Service Issue  leaving over 20 minute...
14637      Please bring American Airlines to #BlackBerry10
14638    Customer Service Issue  you have my money, you...
14639      we have 8 ppl so we need 2 know how many sea...
Name: text, Length: 14640, dtype: object

In [11]:
def remove_numbers(text):
    words = text
    words = re.findall(r"[^\W\d_]+|\d+", words)
    return " ".join(words)


tweets["text"] = tweets["text"].apply(lambda x: remove_numbers(x))
tweets.text

0                                                What said
1        plus you have added commercials to the experie...
2        I did not today Must mean I need to take anoth...
3        Bad Flight it is really aggressive to blast ob...
4        Ca not Tell and it is a really big bad thing a...
                               ...                        
14635    thank you we got on a different flight to Chicago
14636    Customer Service Issue leaving over 20 minutes...
14637      Please bring American Airlines to BlackBerry 10
14638    Customer Service Issue you have my money you c...
14639    we have 8 ppl so we need 2 know how many seats...
Name: text, Length: 14640, dtype: object

In [12]:
def unique_char(rep, text):
    substitute = re.sub(r"(\w)\1+", rep, text)
    return substitute


def char(text):
    substitute = re.sub(r"[^a-zA-Z]", " ", text)
    return substitute


tweets.text

0                                                What said
1        plus you have added commercials to the experie...
2        I did not today Must mean I need to take anoth...
3        Bad Flight it is really aggressive to blast ob...
4        Ca not Tell and it is a really big bad thing a...
                               ...                        
14635    thank you we got on a different flight to Chicago
14636    Customer Service Issue leaving over 20 minutes...
14637      Please bring American Airlines to BlackBerry 10
14638    Customer Service Issue you have my money you c...
14639    we have 8 ppl so we need 2 know how many seats...
Name: text, Length: 14640, dtype: object

In [13]:
def continuosly_repeated_char(text):
    tchr = text.group(0)

    if len(tchr) > 1:
        return tchr[0:2]


tweets["text"] = tweets["text"].apply(
    lambda x: unique_char(continuosly_repeated_char, x)
)
tweets["text"] = tweets["text"].apply(lambda x: char(x))
tweets.text

0                                                What said
1        plus you have added commercials to the experie...
2        I did not today Must mean I need to take anoth...
3        Bad Flight it is really aggressive to blast ob...
4        Ca not Tell and it is a really big bad thing a...
                               ...                        
14635    thank you we got on a different flight to Chicago
14636    Customer Service Issue leaving over    minutes...
14637      Please bring American Airlines to BlackBerry   
14638    Customer Service Issue you have my money you c...
14639    we have   ppl so we need   know how many seats...
Name: text, Length: 14640, dtype: object

In [14]:
tweets["text"] = tweets["text"].apply(lambda x: x.lower())
tweets.text

0                                                what said
1        plus you have added commercials to the experie...
2        i did not today must mean i need to take anoth...
3        bad flight it is really aggressive to blast ob...
4        ca not tell and it is a really big bad thing a...
                               ...                        
14635    thank you we got on a different flight to chicago
14636    customer service issue leaving over    minutes...
14637      please bring american airlines to blackberry   
14638    customer service issue you have my money you c...
14639    we have   ppl so we need   know how many seats...
Name: text, Length: 14640, dtype: object

In [15]:
def remove_stopwords(text):
    text = " ".join(
        [word for word in text.split() if word not in (stopwords.words("english"))]
    )
    return text


tweets["text"] = tweets["text"].apply(lambda x: remove_stopwords(x))
tweets.text

0                                                     said
1                  plus added commercials experience tacky
2                   today must mean need take another trip
3        bad flight really aggressive blast obnoxious e...
4                             ca tell really big bad thing
                               ...                        
14635                   thank got different flight chicago
14636    customer service issue leaving minutes late fl...
14637            please bring american airlines blackberry
14638    customer service issue money change flight ans...
14639    ppl need know many seats next flight plz put u...
Name: text, Length: 14640, dtype: object

In [16]:
X = tweets["text"]
y = tweets["airline_sentiment"]

tfid = TfidfVectorizer()
X_final = tfid.fit_transform(X)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.2, random_state=3
)

In [19]:
smote = SMOTE()
x_sm, y_sm = smote.fit_resample(X_train, y_train)

In [27]:
y_test.value_counts()

0    1793
1     650
2     485
Name: airline_sentiment, dtype: int64

In [20]:
nb = MultinomialNB()
nb.fit(x_sm, y_sm)
nb_prediction = nb.predict(X_test)
print(accuracy_score(nb_prediction, y_test))
print(classification_report(y_test, nb_prediction))

cm = confusion_matrix(y_test, nb_prediction)
cm

0.837431693989071
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1793
           1       0.77      0.57      0.66       650
           2       0.73      0.74      0.74       485

    accuracy                           0.84      2928
   macro avg       0.80      0.76      0.77      2928
weighted avg       0.83      0.84      0.83      2928



array([[1722,   46,   25],
       [ 174,  370,  106],
       [  63,   62,  360]], dtype=int64)

In [88]:
from sklearn.model_selection import cross_val_score

cv_scores_nb = cross_val_score(nb, x_sm, y_sm, cv=10)
print(cv_scores_nb)
np.average(cv_scores_nb)

[0.82679739 0.85984023 0.83405955 0.84204793 0.85434072 0.8732292
 0.87722485 0.88921177 0.87940429 0.89248093]


0.8628636852488907

In [21]:
svm = LinearSVC(C=15)
svm.fit(x_sm, y_sm)
svm_prediction = svm.predict(X_test)
print(accuracy_score(svm_prediction, y_test))
print(classification_report(y_test, svm_prediction))

cm_svm = confusion_matrix(y_test, svm_prediction)
cm_svm

0.8688524590163934
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      1793
           1       0.73      0.72      0.72       650
           2       0.67      0.69      0.68       485

    accuracy                           0.87      2928
   macro avg       0.79      0.79      0.79      2928
weighted avg       0.87      0.87      0.87      2928



array([[1745,   38,   10],
       [  32,  465,  153],
       [  18,  133,  334]], dtype=int64)

In [93]:
cv_scores_svm = cross_val_score(LinearSVC(max_iter=10000, C=15), x_sm, y_sm, cv=5)
print(cv_scores_svm)
np.average(cv_scores_svm)

[0.95024514 0.935355   0.9522426  0.95460323 0.95713767]


0.949916729168985

In [46]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(
    LinearSVC(max_iter=10000),
    {"C": [1, 10, 15, 20, 30]},
    cv=5,
    return_train_score=False,
)

clf.fit(x_sm, y_sm)
df = pd.DataFrame(clf.cv_results_)

In [47]:
df[["param_C", "mean_test_score"]]

,param_C,mean_test_score
0,15,0.949880
1,16,0.949917
2,17,0.949735
3,40,0.948682
4,100,0.946648


In [22]:
lr = LogisticRegression(max_iter=1000, C=30)
lr.fit(x_sm, y_sm)
lr_prediction = lr.predict(X_test)
print(accuracy_score(lr_prediction, y_test))
print(classification_report(y_test, lr_prediction))

cm_lr = confusion_matrix(y_test, lr_prediction)
cm_lr

0.8866120218579235
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1793
           1       0.77      0.76      0.76       650
           2       0.70      0.70      0.70       485

    accuracy                           0.89      2928
   macro avg       0.82      0.81      0.81      2928
weighted avg       0.89      0.89      0.89      2928



array([[1764,   21,    8],
       [  19,  493,  138],
       [  18,  128,  339]], dtype=int64)

In [62]:
clf_lr = GridSearchCV(
    LogisticRegression(max_iter=10000),
    {
        "C": [1, 10, 20, 30],
    },
    cv=5,
    return_train_score=False,
)

clf_lr.fit(x_sm, y_sm)
df_lr = pd.DataFrame(clf_lr.cv_results_)

In [63]:
df_lr[["param_C", "mean_test_score"]]

,param_C,mean_test_score
0,25,0.949989
1,30,0.951587


In [24]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(x_sm, y_sm)
rfc_prediction = rfc.predict(X_test)
print(accuracy_score(rfc_prediction, y_test))
print(classification_report(y_test, rfc_prediction))

cm_rfc = confusion_matrix(y_test, rfc_prediction)
cm_rfc

0.9122267759562842
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      1793
           1       0.83      0.81      0.82       650
           2       0.80      0.73      0.76       485

    accuracy                           0.91      2928
   macro avg       0.87      0.85      0.86      2928
weighted avg       0.91      0.91      0.91      2928



array([[1789,    4,    0],
       [  34,  527,   89],
       [  25,  105,  355]], dtype=int64)

In [83]:
clf_rf = GridSearchCV(
    RandomForestClassifier(),
    {
        "n_estimators": [100, 200, 300],
    },
    cv=5,
    return_train_score=False,
)

clf_rf.fit(x_sm, y_sm)
df_rf = pd.DataFrame(clf_rf.cv_results_)

In [84]:
df_rf[["param_n_estimators", "mean_test_score"]]

,param_n_estimators,mean_test_score
0,100,0.960050
1,200,0.960921
2,300,0.961284


In [94]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_prediction = dt.predict(X_test)
print(accuracy_score(dt_prediction, y_test))
print(classification_report(y_test, dt_prediction))

cm_dt = confusion_matrix(y_test, dt_prediction)
cm_dt

0.9464318140548393
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1913
           1       0.92      0.92      0.92      1783
           2       0.92      0.93      0.93      1811

    accuracy                           0.95      5507
   macro avg       0.95      0.95      0.95      5507
weighted avg       0.95      0.95      0.95      5507



array([[1885,   14,   14],
       [  17, 1639,  127],
       [   3,  120, 1688]], dtype=int64)

In [25]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_sm, y_sm)
knn_prediction = knn.predict(X_test)
print(accuracy_score(knn_prediction, y_test))
print(classification_report(y_test, knn_prediction))

cm_knn = confusion_matrix(y_test, knn_prediction)
cm_knn

0.3203551912568306
              precision    recall  f1-score   support

           0       0.97      0.08      0.14      1793
           1       0.27      0.77      0.40       650
           2       0.31      0.62      0.42       485

    accuracy                           0.32      2928
   macro avg       0.52      0.49      0.32      2928
weighted avg       0.70      0.32      0.25      2928



array([[ 139, 1141,  513],
       [   4,  498,  148],
       [   1,  183,  301]], dtype=int64)

In [99]:
clf_knn = GridSearchCV(
    KNeighborsClassifier(),
    {
        "n_neighbors": [1, 4, 5],
    },
    cv=5,
    return_train_score=False,
)

clf_knn.fit(x_sm, y_sm)
df_knn = pd.DataFrame(clf_knn.cv_results_)

In [100]:
df_knn[["param_n_neighbors", "mean_test_score"]]

,param_n_neighbors,mean_test_score
0,2,0.639028
1,3,0.625118
2,4,0.606741


In [ ]:
nn = MLPClassifier(
    hidden_layer_sizes=(150, 100, 50), max_iter=300, activation="relu", solver="adam"
)
nn.fit(X_train, y_train)
nn_prediction = nn.predict(X_test)
print(accuracy_score(nn_prediction, y_test))
print(classification_report(y_test, nn_prediction))

cm_nn = confusion_matrix(y_test, knn_prediction)
cm_nn